In [1]:
# Import libraries
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Stock_Price_Prediction/data/TATASTEEL.csv')

data = data[['Date', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume']]

data

,Date,Open,High,Low,Close,VWAP,Volume
0,2000-01-03,148.00,153.20,146.10,152.45,150.92,2003185
1,2000-01-04,150.10,153.00,143.05,150.80,151.03,1555136
2,2000-01-05,144.60,162.90,144.60,156.55,156.85,3840284
3,2000-01-06,158.95,169.10,158.95,168.25,167.61,2560449
4,2000-01-07,173.40,179.00,166.30,171.95,173.89,3641691
...,...,...,...,...,...,...,...
5301,2021-04-26,935.00,956.00,930.05,940.75,942.98,21234858
5302,2021-04-27,948.30,983.00,944.30,977.75,965.43,24904515
5303,2021-04-28,985.00,986.00,962.00,971.40,972.08,20447968
5304,2021-04-29,983.00,1036.95,983.00,1031.35,1015.76,44718647


In [7]:
# Features construction
data['Open-Close'] = (data.Open - data.Close)/data.Open
data['High-Low'] = (data.High - data.Low)/data.Low
data['percent_change'] = data['VWAP'].pct_change()
data['std_5'] = data['percent_change'].rolling(5).std()
data['ret_5'] = data['percent_change'].rolling(5).mean()
data.dropna(inplace=True)

# X is the input variable
X = data[['Open-Close', 'High-Low', 'std_5', 'ret_5']]

# Y is the target or output variable
y = np.where(data['VWAP'].shift(-1) > data['VWAP'], 1, -1)

In [8]:
# Total dataset length
dataset_length = data.shape[0]

# Training dataset length
split = int(dataset_length * 0.75)
split

3975

In [9]:
# Splitiing the X and y into train and test datasets
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Print the size of the train and test dataset
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(3975, 4) (1326, 4)
(3975,) (1326,)


In [10]:
# Create the model on train dataset
clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
print('Correct Prediction (%): ', accuracy_score(y_test, model.predict(X_test), normalize=True)*100.0)

# Run the code to view the classification report metrics
from sklearn.metrics import classification_report
report = classification_report(y_test, model.predict(X_test))
print(report)

Correct Prediction (%):  62.06636500754148
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60       630
           1       0.64      0.64      0.64       696

    accuracy                           0.62      1326
   macro avg       0.62      0.62      0.62      1326
weighted avg       0.62      0.62      0.62      1326

